# 🚀 Deploy a Trained Policy

This notebook guides you through deploying a trained policy to a physical robot. 

### Process:
1.  **Configure**: Set the path to your trained model and the robot's server address.
2.  **Load**: The `policy_loader` automatically loads the model and its training configuration.
3.  **Deploy**: The `deploy_policy` function starts the inference loop and sends commands to the robot.

The deployment script automatically handles details like the action space (`tcp`, `joint`, etc.) based on the loaded training configuration.

## 1. Configuration

First, specify the necessary parameters for deployment. **You must edit these values.**

In [ ]:
import pathlib

# TODO: Change to the directory containing your trained policy checkpoint.
# Example: "outputs/2025-09-14/12-00-00"
CHECKPOINT_DIR = pathlib.Path("outputs/<policy_checkpoint_dir>")

# TODO: Change to the robot's IP address.
SERVER_ENDPOINT = "<robot_ip_address>:50051"

# Inference frequency in Hz. Higher values result in smoother but potentially faster movements.
INFERENCE_FREQUENCY_HZ: float = 5.0

print(f"Attempting to load policy from: {CHECKPOINT_DIR}")
print(f"Robot server endpoint: {SERVER_ENDPOINT}")
print(f"Inference frequency: {INFERENCE_FREQUENCY_HZ} Hz")

## 2. Load the Policy

Now, we load the policy from the specified checkpoint directory. The loader will find the latest checkpoint and its corresponding configuration file.

In [ ]:
import torch
from example_policies.robot_deploy.deploy_core.policy_manager import PolicyManager

# Select device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the policy bundle (includes policy, config, translator, printer)
policy_bundle = PolicyManager.load_single(CHECKPOINT_DIR, device)

print("✅ Policy loaded successfully!")
print(f"   Device: {device}")
print(f"   Termination signal support: {policy_bundle.has_termination}")

## 3. Verify Policy Configuration

Verify the loaded policy configuration matches your expectations.

In [ ]:
# Verify loaded policy configuration
cfg = policy_bundle.config

print(f"Policy type: {cfg.type}")
print(f"Input features: {list(cfg.input_features.keys())}")
print(f"Output features: {list(cfg.output_features.keys())}")
print(f"Action shape: {cfg.output_features['action'].shape}")

if hasattr(cfg, 'metadata') and cfg.metadata:
    print(f"\nAction names: {cfg.metadata['features']['action']['names']}")
    print(f"State names: {cfg.metadata['features']['observation.state']['names']}")

## 4. Deploy to Robot

Finally, execute the cell below to start sending commands to the robot.

⚠️ **Warning**: This will move the physical robot. Ensure the robot has a clear and safe workspace.

In [ ]:
from example_policies.robot_deploy.deploy_core.deployment_structures import InferenceConfig
from example_policies.robot_deploy.deploy_core.inference_runner import InferenceRunner
from example_policies.robot_deploy.robot_io.connection import RobotConnection
from example_policies.robot_deploy.robot_io.robot_interface import RobotClient, RobotInterface

# Setup inference configuration
config = InferenceConfig(
    hz=INFERENCE_FREQUENCY_HZ,
    device=device,
    controller=RobotClient.CART_WAYPOINT,  # Most stable and responsive
)

# Run inference loop
print("Starting inference loop...")
print(f"  Frequency: {INFERENCE_FREQUENCY_HZ} Hz")
print(f"  Controller: CART_WAYPOINT")

with RobotConnection(SERVER_ENDPOINT) as stub:
    robot_interface = RobotInterface(stub, policy_bundle.config)
    runner = InferenceRunner(robot_interface, config)
    
    try:
        while True:
            termination = runner.run_step(policy_bundle)
            # Optionally handle termination signal
            # if termination is not None and termination > 0.5:
            #     print("Termination signal received!")
            #     break
    except KeyboardInterrupt:
        print("\n⏹️ Inference stopped by user")